Table 4.5.1: Disability, new presentations to treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.5.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Disability')
cell.assert_one()
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace()
disability = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(disability,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit',' clients in new treatment'),
            HDimConst('Period', '2018-03-31T00:00:00/P1Y'),
            HDimConst('Basis of treatment','disability')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'] = new_table['Treatment group'].map(
    lambda x: {
        'Total' : 'All' 
        }.get(x, x))

In [8]:
new_table['Clients in treatment'] = new_table['Clients in treatment'].map(
    lambda x: {
        'Total' : 'all clients',
        }.get(x, x))

In [9]:
new_table = new_table[['Period','Basis of treatment','Treatment group','Clients in treatment','Measure Type','Value','Unit']]

In [10]:
new_table.head()

,Period,Basis of treatment,Treatment group,Clients in treatment,Measure Type,Value,Unit
0,2018-03-31T00:00:00/P1Y,disability,Behaviour and emotional,Opiate,Count,4352,clients in new treatment
1,2018-03-31T00:00:00/P1Y,disability,Behaviour and emotional,Non-opiate only,Count,1834,clients in new treatment
2,2018-03-31T00:00:00/P1Y,disability,Behaviour and emotional,Non-opiate and Alcohol,Count,2181,clients in new treatment
3,2018-03-31T00:00:00/P1Y,disability,Behaviour and emotional,Alcohol only,Count,4725,clients in new treatment
4,2018-03-31T00:00:00/P1Y,disability,Behaviour and emotional,all clients,Count,13092,clients in new treatment
